<a href="https://colab.research.google.com/github/swKyungbock/2023MLwithTextData/blob/main/8%EC%9E%A5_%EC%9D%B8%ED%94%84%EB%9F%B0%EC%83%88%ED%95%B4%EB%8B%A4%EC%A7%90%EB%8C%93%EA%B8%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 인프런 2020년 새해 다짐 이벤트 댓글 분석(군집화)


## 라이브러리 로드

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 데이터로드하기

In [ ]:
# df 라는 변수에 이벤트 댓글 파일을 로드합니다.
df = pd.read_csv("https://bit.ly/inflearn-event-text-csv")
df.shape

In [ ]:
df.head()

In [ ]:
# tail 로 일부보기
df.tail()

## 데이터 전처리

### 네트워크 오류 등으로 발생한 중복 입력 값을 제거
* 빈도 수 중복을 방지하기 위해

In [ ]:
# drop_duplicates 를 통해 중복을 제거하며, 이때 마지막 글을 남김
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

### 원본은 따로 보존

In [ ]:
# 전처리 전에 원본을 보존하기 위해 origin_text 라는 컬럼에 복사
df["origin_text"] = df["text"]
df.head()

### 소문자 변환

In [ ]:
# "text" 파이썬은 대소문자를 구분하기 때문에 데이터 필터링을 위해 대문자를 모두 소문자로 변경
df["text"] = df["text"].str.lower()

In [ ]:
# 같은 의미의 단어를 하나로 통일 예) python => 파이썬
# replace 는 텍스트가 완전히 일치될 때만 사용할 수 있음
# 일부만 일치한다면 str.replace 를 사용하면 원하는 텍스트로 변경이 가능
df["text"] = df["text"].str.replace(
    "python", "파이썬").str.replace(
    "pandas", "판다스").str.replace(
    "javascript", "자바스크립트").str.replace(
    "java", "자바").str.replace(
    "react", "리액트")

## 문자열 분리로 관심 강의 분리

In [ ]:
# 이 이벤트에는 "관심강의"라는 텍스트가 있음
# "관심강의"를 기준으로 텍스트를 분리하고 관심강의 뒤에 있는 텍스트를 가져옴
# 대부분 "관심강의"라는 텍스트를 쓰고 뒤에 강의명을 쓰기 때문
# 전처리한 내용은 실수를 방지하기 위해 "course" 라는 새로운 컬럼에 담고
# "관심 강의", "관심 강좌" 에 대해서도 똑같이 전처리
# ":" 특수문자를 빈문자로 변경

df["course"] = df["text"].apply(lambda x: x.split("관심강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강좌")[-1])
df["course"] = df["course"].str.replace(":", "")
df["course"].head()

In [ ]:
# "text", "course" 전처리 내용 미리보기
df[["text", "course"]].head()

## 띄어 쓰기를 제거한 텍스트에서 키워드 추출
* TIOBE 프로그래밍 언어 순위 : [index | TIOBE - The Software Quality Company](https://www.tiobe.com/tiobe-index/?fbclid=IwAR34dJfgDHq2DK0C6X3g8IsUno2NhOiikMyxT6fw9SoyujFhy5FPvQogMoA)

In [ ]:
# 특정 키워드가 들어가는 댓글을 찾음
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', '공공데이터',
                  'django', '크롤링', '시각화', '데이터분석',
                  '웹개발', '엑셀', 'c', '자바', '자바스크립트',
                  'node', 'vue', '리액트']

# for 문을 통해 해당 키워드가 있는지 여부를 True, False값으로 표시하도록 함
# 키워드에 따라 컬럼을 새로 만듭니다.
for keyword in search_keyword:
    df[keyword] = df["course"].str.contains(keyword)

In [ ]:
# 미리보기
df.head()

In [ ]:
# 파이썬|공공데이터|판다스 라는 텍스트가 들어가는 데이터가 있는지 찾음
df_python = df[df["text"].str.contains("파이썬|공공데이터|판다스")].copy()
df_python.shape

In [ ]:
# 결과를 모두 더하면 해당 키워드의 등장 빈도수를 카운트
# search_keyword 컬럼만 가져와서 빈도수를 sum으로 합계를 구함
df[search_keyword].sum().sort_values(ascending=False)

In [ ]:
# 공공데이터 텍스트가 들어가는 문장만 찾음
# pandas 를 통해 볼때 문장이 길면 끝까지 보이지 않음
# 문장의 전체를 보기 위해 for문을 통해 해당 텍스트를 순회하며 출력
# 이 때, 데이터 사이에 ------ 줄로 구분해서 표시하도록 함
text = df.loc[(df["공공데이터"] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

## 판다스 단어가 들어가는 텍스트만 찾기
* 이미 str.contains 를 통해 판다스가 들어가는 텍스트에 대해 컬럼을 만들어 놨습니다. 이 값이  True 라면 판다스 강좌 입니다.

In [ ]:
# pandas 라는 텍스트가 들어가는 내용만 찾음

df.loc[df["판다스"] == True, "text"]

## 빈도수 계산을 위한 텍스트 데이터 벡터화
* BOW 단어 가방에 단어를 토큰화 해서 담아줌

In [ ]:
# split으로 "파이썬 데이터 분석" 이라는 텍스트를 토큰화
"파이썬 데이터 분석".split()

In [ ]:
# 사이킷런의 CountVectorizer 를 통해 벡터화
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있음
    tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있음
    preprocessor = None, # 전처리 도구
    stop_words = None, # 불용어 nltk등의 도구를 사용할 수도 있음
    min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋음
    ngram_range=(3, 6), # BOW의 단위 갯수의 범위를 지정
    max_features = 2000 # 만들 피처의 수, 단어의 수
    )
vectorizer

In [ ]:
# df['course'] 만 벡터화
feature_vector = vectorizer.fit_transform(df['course'])
feature_vector.shape

In [ ]:
# vectorizer 에서 get_feature_names 를 추출
vocab = vectorizer.get_feature_names_out()
print(len(vocab))
vocab[:10]

In [ ]:
# 각 리뷰마다 등장하는 단어에 빈도수가 표현됨 0 은 등장하지 않음을 의미
pd.DataFrame(feature_vector[:10].toarray(), columns=vocab).head()

In [ ]:
# 위에서 구한 단어벡터를 더하면 단어가 전체에서 등장하는 횟수를 알 수 있음
# 벡터화 된 피처를 확인해 봄
# Bag of words 에 몇 개의 단어가 들어있는지 확인
dist = np.sum(feature_vector, axis=0)

df_freq = pd.DataFrame(dist, columns=vocab)
df_freq

In [ ]:
# 행과 열의 축을 T로 바꿔주고 빈도수로 정렬
df_freq.T.sort_values(by=0, ascending=False).head(30)

In [ ]:
# ["course", "freq"] 라는 컬럼명을 주어 위에서 만든 데이터프레임을 변환
df_freq_T = df_freq.T.reset_index()
df_freq_T.columns = ["course", "freq"]
df_freq_T.head()

In [ ]:
# 강의명을 토큰 3개로 중복제거하기 위해, 강좌명에서 지식공유자의 이름을 빈문자열로 변경
# 강의명을 lambda 식을 사용해서 x.split() 으로 나누고 [:4] 앞에서 4개까지만 텍스트를 가져오고 다시 join으로 합쳐줌
# 중복된 텍스트를 구분해서 보기 위함
df_freq_T["course_find"] = df_freq_T["course"].str.replace("박조은", "")
df_freq_T["course_find"] = df_freq_T["course_find"].apply(lambda x : " ". join(x.split()[:4]))
df_freq_T.sort_values(["course_find", "freq"], ascending=False).head(10)

In [ ]:
# 3개의 ngram과 빈도수로 역순 정렬을 하게 되면 빈도수가 높고, ngram수가 많은 순으로 정렬이 됨
# 여기에서 drop_duplicates로 첫 번째 강좌를 남기고 나머지 중복을 삭제
print(df_freq_T.shape)
df_course = df_freq_T.drop_duplicates(["course_find", "freq"], keep="first")
print(df_course.shape)

In [ ]:
# 빈도수로 정렬을 하고 어떤 강좌가 댓글에서 가장 많이 언급되었는지 봄
df_course = df_course.sort_values(by="freq", ascending=False)
df_course.head(20)

In [ ]:
# 전처리가 다 되었다면 다른 팀 또는 담당자에게 전달하기 위해 csv 형태로 저장
df_course.to_csv("event-course-name-freq.csv")

## TF-IDF 로 가중치를 주어 벡터화
### TfidfTransformer()


In [ ]:
# TfidfTransformer 를 불러와서 가중치를 주어 벡터화
# transformer 라는 변수로 저장하고 재사용합니다.
from sklearn.feature_extraction.text import TfidfTransformer
tfidftrans = TfidfTransformer(smooth_idf=False)
tfidftrans

In [ ]:
feature_tfidf = tfidftrans.fit_transform(feature_vector)
feature_tfidf.shape

In [ ]:
tfidf_freq = pd.DataFrame(feature_tfidf.toarray(), columns=vocab)
tfidf_freq.head()

In [ ]:
df_tfidf = pd.DataFrame(tfidf_freq.sum())
df_tfidf_top = df_tfidf.sort_values(by=0, ascending=False)
df_tfidf_top.head(10)

In [ ]:
# 중간에 생략되는 단어를 자세히 보고자 할 때
for t in df_tfidf_top.index[:30]:
    print(t)

## 군집화
* 실루엣 분석추가 https://www.kaggle.com/fabiendaniel/customer-segmentation
### KMeans

In [ ]:
from sklearn.cluster import KMeans
from tqdm import trange
inertia = []

start = 10
end = 70

# 적절한 클러스터의 갯수를 알기 위해 inertia 값을 구함
# trange 를 통해 시작과 끝 값을 지정해 주면 진행 정도를 알 수 있습니다.
# 학습을 할 때는 feature_tfidf 값을 사용합니다.
# cls.inertia_ 값을 inertia 리스트에 저장합니다.
for i in trange(start, end):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(feature_tfidf)
    inertia.append(kmeans.inertia_)

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import koreanize_matplotlib
#그래프에 retina display 적용
%config InlineBackend.figure_format='retina'

In [ ]:
# 위에서 구한 값을 시각화
# x축에는 클러스터의 수를 y축에는 inertia 값을 넣어 그림

plt.plot(range(start, end), inertia)
plt.title("KMeans 클러스터 수 비교")

* 적정한 클러스터 갯수를 넣어 군집화 합니다.

In [ ]:
# n_clusters 에 적절한 값을 넣어줌
# fit.predict 를 하고 결과를 cluster 라는 새로운 컬럼에 담음
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(feature_tfidf)
prediction = kmeans.predict(feature_tfidf)
df["cluster"] = prediction

In [ ]:
# df["cluster"] 의 빈도수를 value_counts로 세어봅니다.
df["cluster"].value_counts().head(10)

### MiniBatchKMeans
* [Comparison of the K-Means and MiniBatchKMeans clustering algorithms — scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/cluster/plot_mini_batch_kmeans.html)

In [ ]:
# batch_size 를 쓸 수 있는 MiniBatchKMeans 로 군집화
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
b_inertia = []
silhouettes = []

# 적절한 클러스터의 갯수를 알기 위해 inertia 값을 구함
# trange 를 통해 시작과 끝 값을 지정해 주면 진행 상태를 확인 할 수 있음
# b_inertia 리스트에 cls.inertia_ 값을 넣어줍니다.
for i in trange(start, end):
    mkmeans = MiniBatchKMeans(n_clusters=i, random_state=42)
    mkmeans.fit(feature_tfidf)
    b_inertia.append(mkmeans.inertia_)
    silhouettes.append(silhouette_score(feature_tfidf, mkmeans.labels_))

In [ ]:
# 위에서 구한 값을 시각화 합니다.
# x축에는 클러스터의 수를 y축에는 b_inertia 값을 넣어 시각화
plt.plot(range(start, end), b_inertia)
plt.title("MiniBatchKMeans 클러스터 수 비교")

In [ ]:
plt.figure(figsize=(15, 4))
plt.title('Silhouette Score')
plt.plot(range(start, end), silhouettes)
plt.xticks(range(start, end))
plt.show()

In [ ]:
# yellowbrick 은 머신러닝 시각화 도구로 별도의 설치가 필요
!pip install yellowbrick

In [ ]:
# yellowbrick.cluster 에서 KElbowVisualizer 불러오기
from yellowbrick.cluster import KElbowVisualizer

KElbowM = KElbowVisualizer(kmeans, k=(start, end))
KElbowM.fit(feature_tfidf.toarray())
KElbowM.show()

In [ ]:
# MiniBatchKMeans 를 통해 학습
# 결과를 bcluster 라는 변수에 저장
mkmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
mkmeans.fit(feature_tfidf)
prediction = mkmeans.predict(feature_tfidf)
df["bcluster"] = prediction

In [ ]:
# bcluster의 빈도수
df["bcluster"].value_counts().head(10)

In [ ]:
# 어떤 강좌명이 있는지 특정 클러스터의 값을 봄
df.loc[df["bcluster"] == 21, "course"].value_counts().head(1)

In [ ]:
df.loc[df["bcluster"] == 21, ["bcluster", "cluster", "course"]]

In [ ]:
df.loc[df["bcluster"] == 24, ["bcluster", "cluster", "origin_text", "course"]].tail(10)

### 클러스터 예측 평가하기

In [ ]:
# n_clusters 위에서 정의한 클러스터 수를 사용
feature_array = feature_vector.toarray()
# 예측한 클러스터의 유니크 값
labels = np.unique(prediction)
df_cluster_score = []
df_cluster = []
for label in labels:
    id_temp = np.where(prediction==label) # 예측한 값이 클러스터 번호와 매치 되는 것을 가져옴
    x_means = np.mean(feature_array[id_temp], axis = 0) # 클러스터의 평균 값을 구함
    sorted_means = np.argsort(x_means)[::-1][:n_clusters] # 값을 역순으로 정렬해서 클러스터 수 만큼 가져옴
    features = vectorizer.get_feature_names_out()
    best_features = [(features[i], x_means[i]) for i in sorted_means]
    # 클러스터별 전체 스코어
    df_score = pd.DataFrame(best_features, columns = ['features', 'score'])
    df_cluster_score.append(df_score)
    # 클러스터 대표 키워드
    df_cluster.append(best_features[0])

In [ ]:
# 개별 클러스터에서 점수가 가장 높은 단어를 추출 아래 점수가 클수록 예측 정확도가 높음
# MiniBatchKMeans 로 예측한 값 기준
pd.DataFrame(df_cluster, columns = ['features', 'score']).sort_values(by=["features", "score"], ascending=False)

In [ ]:
# score 정확도가 1이 나온 클러스터를 찾아봄 - 같은 강좌끼리 묶였는지 확인 함
df.loc[df["bcluster"] == 28, ["bcluster", "cluster", "origin_text", "course"]]

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer
visualizer = SilhouetteVisualizer(mkmeans, colors='yellowbrick')

visualizer.fit(feature_tfidf.toarray())
visualizer.show()